<a href="https://colab.research.google.com/github/ea5055/nlp3/blob/master/event3_bakus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Cup Event #3

In [0]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.22.2.post1.


In [0]:
import os
os.getcwd()

'/content'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH = 'drive/My Drive/Colab Notebooks/'

In [0]:
df = pd.read_csv(os.path.join(PATH, "spamraw_train.csv"))
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   sms_text  5000 non-null   object
 2   spam      5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


,id,sms_text,spam
0,1,Hope you are having a good week. Just checking in,0
1,2,K..give back my thanks.,0
2,3,Am also doing in cbe only. But have to pay.,0
3,4,"complimentary 4 STAR Ibiza Holiday or £10,000 ...",1
4,5,okmail: Dear Dave this is your final notice to...,1


In [0]:
# ensemble

import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "ensemble",
                "ngram_size" : 4,
                "attr":"lower"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [0]:
from sklearn.model_selection import train_test_split

X = df['sms_text']
y = df['spam']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
X_train.head()
y_train.head()
X_val.head()
y_val.head()

4227    Cashbin.co.uk (Get lots of cash this weekend!)...
4676         Once free call me sir. I am waiting for you.
800     Keep yourself safe for me because I need you a...
3671    You call him and tell now infront of them. Cal...
4193                    I reach home safe n sound liao...
Name: sms_text, dtype: object

4227    1
4676    0
800     0
3671    0
4193    0
Name: spam, dtype: int64

1501        And he's apparently bffs with carly quick now
2586                       No problem. How are you doing?
2653                               Y de asking like this.
1055    07732584351 - Rodger Burns - MSG = We tried to...
705     Lol! U drunkard! Just doing my hair at d momen...
Name: sms_text, dtype: object

1501    0
2586    0
2653    0
1055    1
705     0
Name: spam, dtype: int64

In [0]:
# Add labels to text classifier
textcat.add_label("ham")
textcat.add_label("spam")

1

1

In [0]:
train_texts = X_train.values
train_labels = [{'cats': { 'ham' : spam == 0 ,
                           'spam' : spam == 1 }} 
                for spam in y_train]

In [0]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Cashbin.co.uk (Get lots of cash this weekend!) www.cashbin.co.uk Dear Welcome to the weekend We have got our biggest and best EVER cash give away!! These..',
  {'cats': {'ham': False, 'spam': True}}),
 ('Once free call me sir. I am waiting for you.',
  {'cats': {'ham': True, 'spam': False}}),
 ("Keep yourself safe for me because I need you and I miss you already and I envy everyone that see's you in real life",
  {'cats': {'ham': True, 'spam': False}})]

In [0]:
import random
from spacy.util import minibatch

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(1):  #was 30
    random.shuffle(train_data)
    # Create the batch generator with batch size = 5
    batches = minibatch(train_data, size=5)
    # Iterate through minibatches
    for batch in batches:
        # Each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # This is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.4286070488141401}


In [0]:
texts = X_val 

docs = [nlp.tokenizer(text) for text in texts]
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.9275982e-01 7.2401869e-03]
 [9.9980718e-01 1.9278844e-04]
 [9.9909937e-01 9.0065692e-04]
 ...
 [9.9939585e-01 6.0414831e-04]
 [9.7878540e-01 2.1214548e-02]
 [9.9931717e-01 6.8281544e-04]]


In [0]:
# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)


In [0]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

pred_val = predicted_labels

print ("Confusion matrix:")
print(confusion_matrix(y_val, pred_val))

print("\nF1 Score = {:.2f}".format(f1_score(y_val, pred_val)))
print(classification_report(y_val, pred_val))

print("\nF1 Score (Kaggle) = {:.6f}".format(f1_score(y_val, pred_val, average='micro')))


Confusion matrix:
[[868   2]
 [  9 121]]

F1 Score = 0.96
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       870
           1       0.98      0.93      0.96       130

    accuracy                           0.99      1000
   macro avg       0.99      0.96      0.98      1000
weighted avg       0.99      0.99      0.99      1000


F1 Score (Kaggle) = 0.989000


F1 Score = 0.96
F1 Score (Kaggle) = 0.989000
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       870
           1       0.98      0.93      0.96       130

    accuracy                           0.99      1000
   macro avg       0.99      0.96      0.98      1000
weighted avg       0.99      0.99      0.99      1000



In [0]:
test_df = pd.read_csv(os.path.join(PATH, "spamraw_test.csv"))

In [0]:
texts_test = test_df['sms_text']

docs_test = [nlp.tokenizer(text) for text in texts_test]
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
test_scores, _ = textcat.predict(docs_test)

print(test_scores)

[[1.49236395e-04 9.99850750e-01]
 [9.99997973e-01 2.03671993e-06]
 [9.99553978e-01 4.45978745e-04]
 ...
 [1.09682485e-04 9.99890327e-01]
 [7.96455801e-01 2.03544155e-01]
 [9.99839067e-01 1.60943717e-04]]


In [0]:
# From the scores, find the label with the highest score/probability
predicted_labels_test = test_scores.argmax(axis=1)


In [0]:
predicted_labels_test

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,

In [0]:
my_submission = pd.DataFrame({'id': test_df.id, 'predicted': predicted_labels_test})

my_submission.head()

# you could use any filename. We choose submission here
my_submission.to_csv('submission-bakus.csv', index=False)

,id,predicted
0,12000,1
1,12001,0
2,12002,0
3,12003,0
4,12004,0


In [0]:
pred_test = predicted_labels_test

In [0]:
import pickle
pickle.dump(pred_test, open(os.path.join(PATH, "pred_test_backus.pickle"), "wb"))

In [0]:
#Solution - Kaggle: 99.105 | 98.7478
df_sol = pd.read_csv(os.path.join(PATH, "spamraw_test_solutions.csv"))
y_sol = df_sol['spam']

print("Solution:")
print("---------")
print("F1 Score = {:.6f}".format(f1_score(y_sol, pred_test, average='micro')))
print()
print(classification_report(y_sol, pred_test))
confusion_matrix(y_sol, pred_test)

Solution:
---------
F1 Score = 0.987478

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       485
           1       1.00      0.91      0.95        74

    accuracy                           0.99       559
   macro avg       0.99      0.95      0.97       559
weighted avg       0.99      0.99      0.99       559



array([[485,   0],
       [  7,  67]])